In [49]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [50]:
from machine import Pin
pup = Pin(12, Pin.OUT)
pdown = Pin(13, Pin.OUT)


In [62]:
pdown.value(0)

In [41]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.9.4-631-g338635ccc on 2018-10-11; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nPYB: soft reboot\r\nOSError: [Errno 2] ENOENT\r\nMicroPython v1.9.4-631-g338635ccc on 2018-10-11; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [63]:
# This requires you to checkout https://github.com/micropython/micropython-lib and avoids the upip
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio/uasyncio/__init__.py lib/uasyncio/
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.core/uasyncio/core.py lib/uasyncio/
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.queues/uasyncio/queues.py lib/uasyncio/
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.synchro/uasyncio/synchro.py lib/uasyncio/

# This requires you to checkout https://github.com/peterhinch/micropython-async
%sendtofile --source /home/julian/extrepositories/micropython-async/asyn.py
%sendtofile --source /home/julian/extrepositories/micropython-async/aswitch.py

# This requires you to checkout https://github.com/peterhinch/micropython-mqtt/tree/master/mqtt_as
%sendtofile --source /home/julian/extrepositories/micropython-mqtt/mqtt_as/mqtt_as.py

Sent 638 lines (23042 bytes) to mqtt_as.py.


In [64]:
# Set up the wifi connection
%sendtofile lconfig.py

from mqtt_as import config, ESP32

# Include any cross-project settings.
mqttlocal = "10.0.31.119"

if ESP32:
    config['ssid'] = 'DoESLiverpool'  # EDIT if you're using ESP32
    config['wifi_pw'] = 'decafbad00'
    config['mqttchannel'] = "garageshutter"


Sent 9 lines (258 bytes) to lconfig.py.


In [48]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
[Timed out waiting for recognizable response]
Disconnecting [paste mode not working]
  (You may need to reset the device)

In [65]:
%sendtofile main.py

from mqtt_as import MQTTClient
from lconfig import config, mqttlocal
import uasyncio as asyncio
import asyn
import os
from machine import Pin

# The callback has to be a normal function, but we can create an async task from within it using aloop
def callback(topic, msg):
    aloop.create_task(acallback(topic, msg))

async def conn_han(client):
    print("subscribing", (mqttchannel+'/code'))
    await client.subscribe(mqttchannel+'/code', 1)

config['subs_cb'] = callback
config['connect_coro'] = conn_han
config['server'] = mqttlocal

MQTTClient.DEBUG = True  # Optional: print diagnostic messages
client = MQTTClient(config)
aloop = asyncio.get_event_loop()

mqttchannel = config["mqttchannel"]
#client, aloop = None, None

p12 = Pin(12, Pin.OUT)
p13 = Pin(13, Pin.OUT)

@asyn.cancellable
async def setloop(pin, secs):
    try:
        await client.publish(mqttchannel+'/setval', str(pin)+"=1")
        pin.value(1)
        await asyn.sleep(secs)
        await client.publish(mqttchannel+'/setval', str(pin)+"=0")
        pin.value(0)
    except Exception as e:
        pin.value(0)
        await client.publish(mqttchannel+'/rec', 'error '+str(e))

        
async def acallback(topic, msg):
    print((topic, msg))
    await asyn.Cancellable.cancel_all()  # stop any threads already waiting
    p12.value(0); p13.value(0)   # just in case
    vmsg = msg.split()
    try:
        secs = float(vmsg[1])
    except:
        secs = 1.0
    pin = p13 if vmsg[0].lower() == b"up" else p12
    await client.publish(mqttchannel+'/rec', str(pin)+" secs="+str(secs))
    aloop.create_task(asyn.Cancellable(setloop, pin, secs)())

    
async def main(client):
    await client.connect()
    n = 0
    while True:
        await asyncio.sleep(5)
        await client.publish(mqttchannel+'/beat', '{}'.format(n), qos = 1)
        n += 1

try:
    aloop.run_until_complete(main(client))
finally:
    client.close()  # Prevent LmacRxBlk:1 errors
    

Sent 69 lines (1940 bytes) to main.py.


In [ ]:
# To control the doors:
# mosquitto_pub  -h mqtt.local -t "garageshutter/code" -m "down 0.9"
# mosquitto_pub  -h mqtt.local -t "garageshutter/code" -m "up 0.9"
